In [1]:
import os
from model import LGBM_w_Feature_Selector 
import logging
import warnings
from sklearn.exceptions import DataConversionWarning,ConvergenceWarning
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import re
import numpy as np
warnings.filterwarnings("ignore", category=DataConversionWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

lgbm_param_grid = {
    "hidden_layer_sizes": [(20),(40),(80),(40,10),(80,20),(80,40),(80,20,10),(80,40,20),(80,40,10)], 
    "activation":['relu','logistic'], 
    "solver":['adam',], 
    "alpha":[0.0001, 0.001, 0.01], 
    "learning_rate_init":[0.001, 0.01], 
    "warm_start":[False], 
    "momentum":[0.9], 
} 

mlp_param_grid ={
    "hidden_layer_sizes": [(20),(40),(80),(40,10),(80,20),(80,40),(80,20,10),(80,40,20),(80,40,10)], 
    "activation":['relu','logistic'], 
    "solver":['adam',], 
    "alpha":[0.0001, 0.001, 0.01], 
    "learning_rate_init":[0.001, 0.01], 
    "warm_start":[False], 
    "momentum":[0.9], 
} 


In [8]:
import scipy.io

file = scipy.io.loadmat('/home/efe/Downloads/madelon.mat')

import pandas as pd

# Extract arrays from the dictionary
X = file['X']
Y = file['Y']

# Create a DataFrame from X
data = pd.DataFrame(X)

# Add Y to the DataFrame
data['y'] = Y
data.columns = data.columns.astype(str)

all_scores = []
mo_scores = []
#data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
#print(data.shape)
nb = len(data)
val_split = 0.3
test_split = 0.1
X_train, X_val, X_test = data.iloc[:int((1-(val_split+test_split))*nb)], data.iloc[int((1-(val_split+test_split))*nb):int((1-(test_split))*nb)], data.iloc[int((1-(test_split))*nb):]
y_train, y_val, y_test = X_train[["y"]], X_val[["y"]], X_test[["y"]]
scaler = MinMaxScaler()
normalized_X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
normalized_X_val = pd.DataFrame(scaler.fit_transform(X_val), columns=X_val.columns)
normalized_X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)
normalized_y_train = pd.DataFrame(scaler.fit_transform(y_train), columns=y_train.columns).values
normalized_y_val = pd.DataFrame(scaler.fit_transform(y_val), columns=y_val.columns).values
normalized_y_test = pd.DataFrame(scaler.fit_transform(y_test), columns=y_test.columns).values
normalized_X_train, normalized_X_val, normalized_X_test = normalized_X_train.drop(columns=['y']).values, normalized_X_val.drop(columns=['y']).values, normalized_X_test.drop(columns=['y']).values
network = LGBM_w_Feature_Selector(model="lgbm",problem_type="regression",param_grid=lgbm_param_grid,X_train=normalized_X_train, X_test=normalized_X_test, slack=0.0, 
                                        X_val=normalized_X_val, y_val=normalized_y_val,y_train=normalized_y_train,y_test=normalized_y_test,iterations=10) 
_,all_score, mo_score = network.feature_extraction(5, seed=42 , method="convergence", loss_tolerance=1,run_CV=True)
all_scores.append(all_score)
mo_scores.append(mo_score)

Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': 80, 'alpha': 0.001, 'activation': 'logistic'}


In [9]:
np.mean(all_scores)

0.1850772065314987

In [10]:
np.mean(mo_scores)

0.18741416398602548

In [11]:
all_scores = []
mo_scores = []
data = pd.read_csv("/home/efe/Downloads/energydata_target_y.csv")
data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
#print(data.shape)
nb = len(data)
val_split = 0.3
test_split = 0.1
X_train, X_val, X_test = data.iloc[:int((1-(val_split+test_split))*nb)], data.iloc[int((1-(val_split+test_split))*nb):int((1-(test_split))*nb)], data.iloc[int((1-(test_split))*nb):]
y_train, y_val, y_test = X_train[["y"]], X_val[["y"]], X_test[["y"]]
scaler = MinMaxScaler()
normalized_X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
normalized_X_val = pd.DataFrame(scaler.fit_transform(X_val), columns=X_val.columns)
normalized_X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)
normalized_y_train = pd.DataFrame(scaler.fit_transform(y_train), columns=y_train.columns).values
normalized_y_val = pd.DataFrame(scaler.fit_transform(y_val), columns=y_val.columns).values
normalized_y_test = pd.DataFrame(scaler.fit_transform(y_test), columns=y_test.columns).values
normalized_X_train, normalized_X_val, normalized_X_test = normalized_X_train.drop(columns=['y']).values, normalized_X_val.drop(columns=['y']).values, normalized_X_test.drop(columns=['y']).values
network = LGBM_w_Feature_Selector(model="lgbm",problem_type="regression",param_grid=lgbm_param_grid,X_train=normalized_X_train, X_test=normalized_X_test, slack=0.0, 
                                        X_val=normalized_X_val, y_val=normalized_y_val,y_train=normalized_y_train,y_test=normalized_y_test,iterations=10) 
_,all_score, mo_score = network.feature_extraction(5, seed=42 , method="convergence", loss_tolerance=1,run_CV=True)
all_scores.append(all_score)
mo_scores.append(mo_score)

Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': 80, 'alpha': 0.001, 'activation': 'logistic'}


In [12]:
np.mean(all_scores)

0.006222426878826595

In [13]:
np.mean(mo_scores)

0.005802037541521015

In [ ]:
#### from this point on, GBMO experiments for LGBM are conducted on M4 hourly

In [14]:
all_scores = []
mo_scores = []
for i in range(200):
    data = pd.read_csv("/home/efe/Documents/m4-dataset/m4-preprocess/m4-preprocess/{}_M4_Hourly.csv".format(str(i)))
    data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    #print(data.shape)
    train_data, X_test = data.iloc[:-50], data.iloc[-50:]
    nb = len(train_data)
    val_split = 0.3
    X_train, X_val = train_data.iloc[:int((1-val_split)*nb)], train_data.iloc[int((1-val_split)*nb):]
    y_train, y_val, y_test = X_train[["y"]], X_val[["y"]], X_test[["y"]]
    scaler = MinMaxScaler()
    normalized_X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    normalized_X_val = pd.DataFrame(scaler.fit_transform(X_val), columns=X_val.columns)
    normalized_X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)
    normalized_y_train = pd.DataFrame(scaler.fit_transform(y_train), columns=y_train.columns).values
    normalized_y_val = pd.DataFrame(scaler.fit_transform(y_val), columns=y_val.columns).values
    normalized_y_test = pd.DataFrame(scaler.fit_transform(y_test), columns=y_test.columns).values
    #X_train=(X_train-X_train.mean())/X_train.std()
    #X_val=(X_val-X_val.mean())/X_val.std()
    #X_test=(X_test-X_test.mean())/X_test.std()
    #y_train=(y_train-y_train.mean())/y_train.std()
    #y_val=(y_val-y_val.mean())/y_val.std()
    #y_test=(y_test-y_test.mean())/y_test.std()
    normalized_X_train, normalized_X_val, normalized_X_test = normalized_X_train.drop(columns=['y']).values, normalized_X_val.drop(columns=['y']).values, normalized_X_test.drop(columns=['y']).values
    network = LGBM_w_Feature_Selector(model="lgbm",problem_type="regression",param_grid=lgbm_param_grid,X_train=normalized_X_train, X_test=normalized_X_test, slack=0.1, 
                                            X_val=normalized_X_val, y_val=normalized_y_val,y_train=normalized_y_train,y_test=normalized_y_test,iterations=10) 
    _,all_score, mo_score = network.feature_extraction(5, seed=42 , method="convergence", loss_tolerance=1,run_CV=True)
    all_scores.append(all_score)
    mo_scores.append(mo_score)

Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': 80, 'alpha': 0.001, 'activation': 'logistic'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': 80, 'alpha': 0.001, 'activation': 'logistic'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': 80, 'alpha': 0.001, 'activation': 'logistic'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': 80, 'alpha': 0.001, 'activation': 'logistic'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hi

In [15]:
np.mean(all_scores)

0.010530135053755347

In [16]:
np.mean(mo_scores)

0.010581905334534014

In [ ]:
### from this point on, GBMO experiments for MLP are conducted on M4 hourly

In [18]:
all_scores = []
mo_scores = []
for i in range(200):
    data = pd.read_csv("/home/efe/Documents/m4-dataset/m4-preprocess/m4-preprocess/{}_M4_Hourly.csv".format(str(i)))
    data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    #print(data.shape)
    train_data, X_test = data.iloc[:-50], data.iloc[-50:]
    nb = len(train_data)
    val_split = 0.3
    X_train, X_val = train_data.iloc[:int((1-val_split)*nb)], train_data.iloc[int((1-val_split)*nb):]
    y_train, y_val, y_test = X_train[["y"]], X_val[["y"]], X_test[["y"]]
    scaler = MinMaxScaler()
    normalized_X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    normalized_X_val = pd.DataFrame(scaler.fit_transform(X_val), columns=X_val.columns)
    normalized_X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)
    normalized_y_train = pd.DataFrame(scaler.fit_transform(y_train), columns=y_train.columns).values
    normalized_y_val = pd.DataFrame(scaler.fit_transform(y_val), columns=y_val.columns).values
    normalized_y_test = pd.DataFrame(scaler.fit_transform(y_test), columns=y_test.columns).values
    #X_train=(X_train-X_train.mean())/X_train.std()
    #X_val=(X_val-X_val.mean())/X_val.std()
    #X_test=(X_test-X_test.mean())/X_test.std()
    #y_train=(y_train-y_train.mean())/y_train.std()
    #y_val=(y_val-y_val.mean())/y_val.std()
    #y_test=(y_test-y_test.mean())/y_test.std()
    normalized_X_train, normalized_X_val, normalized_X_test = normalized_X_train.drop(columns=['y']).values, normalized_X_val.drop(columns=['y']).values, normalized_X_test.drop(columns=['y']).values
    network = LGBM_w_Feature_Selector(model="mlp",problem_type="regression",param_grid=mlp_param_grid,X_train=normalized_X_train, X_test=normalized_X_test, slack=0.1, 
                                            X_val=normalized_X_val, y_val=normalized_y_val,y_train=normalized_y_train,y_test=normalized_y_test,iterations=10) 
    _,all_score, mo_score = network.feature_extraction(5, seed=42 , method="convergence", loss_tolerance=1,run_CV=True)
    all_scores.append(all_score)
    mo_scores.append(mo_score)

Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': (80, 20), 'alpha': 0.01, 'activation': 'relu'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': 80, 'alpha': 0.001, 'activation': 'logistic'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': (80, 20), 'alpha': 0.01, 'activation': 'relu'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': (80, 20), 'alpha': 0.01, 'activation': 'relu'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 

In [19]:
np.mean(all_scores)

0.011122267915681008

In [20]:
np.mean(mo_scores)

0.01067335227410739

In [ ]:
### from this point on, experiments for FLBMO for LGBM on M4 hourly

In [26]:
### try for 3-15 features
all, gb, mo, rfe = {}, {}, {}, {}
param_grid ={
    "hidden_layer_sizes": [(20),(40),(80),(40,10),(80,20),(80,40),(80,20,10),(80,40,20),(80,40,10)], 
    "activation":['relu','logistic'], 
    "solver":['adam',], 
    "alpha":[0.0001, 0.001, 0.01], 
    "learning_rate_init":[0.001, 0.01], 
    "warm_start":[False], 
    "momentum":[0.9], 
} 
for f_number in [3, 5, 8, 12]:
    all_scores = []
    gb_scores = []
    mo_scores = []
    rfe_scores = []
    #f_number = 10
    for i in range(200):
        data = pd.read_csv("/home/efe/Documents/m4-dataset/m4-preprocess/m4-preprocess/{}_M4_Hourly.csv".format(str(i)))
        data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
        #print(data.shape)
        train_data, X_test = data.iloc[:-50], data.iloc[-50:]
        nb = len(train_data)
        val_split = 0.2
        X_train, X_val = train_data.iloc[:int((1-val_split)*nb)], train_data.iloc[int((1-val_split)*nb):]
        y_train, y_val, y_test = X_train[["y"]], X_val[["y"]], X_test[["y"]]
        scaler = MinMaxScaler()
        normalized_X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
        normalized_X_val = pd.DataFrame(scaler.fit_transform(X_val), columns=X_val.columns)
        normalized_X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)
        normalized_y_train = pd.DataFrame(scaler.fit_transform(y_train), columns=y_train.columns).values
        normalized_y_val = pd.DataFrame(scaler.fit_transform(y_val), columns=y_val.columns).values
        normalized_y_test = pd.DataFrame(scaler.fit_transform(y_test), columns=y_test.columns).values
        #X_train=(X_train-X_train.mean())/X_train.std()
        #X_val=(X_val-X_val.mean())/X_val.std()
        #X_test=(X_test-X_test.mean())/X_test.std()
        #y_train=(y_train-y_train.mean())/y_train.std()
        #y_val=(y_val-y_val.mean())/y_val.std()
        #y_test=(y_test-y_test.mean())/y_test.std()
        normalized_X_train, normalized_X_val, normalized_X_test = normalized_X_train.drop(columns=['y']).values, normalized_X_val.drop(columns=['y']).values, normalized_X_test.drop(columns=['y']).values
        network = LGBM_w_Feature_Selector(model="lgbm",problem_type="regression",param_grid=lgbm_param_grid,X_train=normalized_X_train, X_test=normalized_X_test, slack=0.1, 
                                                X_val=normalized_X_val, y_val=normalized_y_val,y_train=normalized_y_train,y_test=normalized_y_test,iterations=10) 
        _,all_score, gb_score, mo_score, rfe_score = network.feature_extraction(f_number, seed=42 , method="number of features", run_CV=True, include_RFE=True)
        all_scores.append(all_score)
        gb_scores.append(gb_score)    
        mo_scores.append(mo_score)
        rfe_scores.append(rfe_score)

all[f_number] = np.mean(all_scores)
gb[f_number] = np.mean(gb_scores)
mo[f_number] = np.mean(mo_scores)
rfe[f_number] = np.mean(rfe_scores)



Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': 80, 'alpha': 0.001, 'activation': 'logistic'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': 80, 'alpha': 0.001, 'activation': 'logistic'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': 80, 'alpha': 0.001, 'activation': 'logistic'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': 80, 'alpha': 0.001, 'activation': 'logistic'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hi

In [27]:
all

{12: 0.010547213591813487}

In [28]:
gb

{12: 0.010349112434579179}

In [29]:
mo

{12: 0.01032394871257196}

In [30]:
rfe

{12: 0.010445958425828288}

In [ ]:
### from this point on, experiments for FLBMO for MLP on M4 hourly

In [ ]:
### try for 3-15 features
all, gb, mo = {}, {}, {}

for f_number in [3, 5, 8, 12]:
    all_scores = []
    gb_scores = []
    mo_scores = []
    #f_number = 10
    for i in range(200):
        data = pd.read_csv("/home/efe/Documents/m4-dataset/m4-preprocess/m4-preprocess/{}_M4_Hourly.csv".format(str(i)))
        data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
        #print(data.shape)
        train_data, X_test = data.iloc[:-50], data.iloc[-50:]
        nb = len(train_data)
        val_split = 0.2
        X_train, X_val = train_data.iloc[:int((1-val_split)*nb)], train_data.iloc[int((1-val_split)*nb):]
        y_train, y_val, y_test = X_train[["y"]], X_val[["y"]], X_test[["y"]]
        scaler = MinMaxScaler()
        normalized_X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
        normalized_X_val = pd.DataFrame(scaler.fit_transform(X_val), columns=X_val.columns)
        normalized_X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)
        normalized_y_train = pd.DataFrame(scaler.fit_transform(y_train), columns=y_train.columns).values
        normalized_y_val = pd.DataFrame(scaler.fit_transform(y_val), columns=y_val.columns).values
        normalized_y_test = pd.DataFrame(scaler.fit_transform(y_test), columns=y_test.columns).values
        #X_train=(X_train-X_train.mean())/X_train.std()
        #X_val=(X_val-X_val.mean())/X_val.std()
        #X_test=(X_test-X_test.mean())/X_test.std()
        #y_train=(y_train-y_train.mean())/y_train.std()
        #y_val=(y_val-y_val.mean())/y_val.std()
        #y_test=(y_test-y_test.mean())/y_test.std()
        normalized_X_train, normalized_X_val, normalized_X_test = normalized_X_train.drop(columns=['y']).values, normalized_X_val.drop(columns=['y']).values, normalized_X_test.drop(columns=['y']).values
        network = LGBM_w_Feature_Selector(model="mlp",problem_type="regression",param_grid=mlp_param_grid,X_train=normalized_X_train, X_test=normalized_X_test, slack=0.1, 
                                                X_val=normalized_X_val, y_val=normalized_y_val,y_train=normalized_y_train,y_test=normalized_y_test,iterations=10) 
        _,all_score, gb_score, mo_score = network.feature_extraction(f_number, seed=42 , method="number of features", run_CV=True)
        all_scores.append(all_score)
        gb_scores.append(gb_score)    
        mo_scores.append(mo_score)
    all[f_number] = np.mean(all_scores)
    gb[f_number] = np.mean(gb_scores)
    mo[f_number] = np.mean(mo_scores)


Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': (80, 20), 'alpha': 0.01, 'activation': 'relu'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': 80, 'alpha': 0.001, 'activation': 'logistic'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': (80, 20), 'alpha': 0.01, 'activation': 'relu'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 'hidden_layer_sizes': (80, 20), 'alpha': 0.01, 'activation': 'relu'}
Best parameters for iteration <built-in function iter> are: {'warm_start': False, 'solver': 'adam', 'momentum': 0.9, 'learning_rate_init': 0.01, 

In [ ]:
all

In [ ]:
gb

In [ ]:
mo

In [ ]:
### from this point on, experiments for GBMO for MLP on M4 daily

In [ ]:
all_scores = []
mo_scores = []
for i in range(200):
    data = pd.read_csv("/home/efe/Documents/m4-dataset/m4-preprocess/m4-preprocess/{}_M4_Daily.csv".format(str(i)))
    data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    #print(data.shape)
    train_data, X_test = data.iloc[:-50], data.iloc[-50:]
    nb = len(train_data)
    val_split = 0.3
    X_train, X_val = train_data.iloc[:int((1-val_split)*nb)], train_data.iloc[int((1-val_split)*nb):]
    y_train, y_val, y_test = X_train[["y"]], X_val[["y"]], X_test[["y"]]
    scaler = MinMaxScaler()
    normalized_X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    normalized_X_val = pd.DataFrame(scaler.fit_transform(X_val), columns=X_val.columns)
    normalized_X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)
    normalized_y_train = pd.DataFrame(scaler.fit_transform(y_train), columns=y_train.columns).values
    normalized_y_val = pd.DataFrame(scaler.fit_transform(y_val), columns=y_val.columns).values
    normalized_y_test = pd.DataFrame(scaler.fit_transform(y_test), columns=y_test.columns).values
    #X_train=(X_train-X_train.mean())/X_train.std()
    #X_val=(X_val-X_val.mean())/X_val.std()
    #X_test=(X_test-X_test.mean())/X_test.std()
    #y_train=(y_train-y_train.mean())/y_train.std()
    #y_val=(y_val-y_val.mean())/y_val.std()
    #y_test=(y_test-y_test.mean())/y_test.std()
    normalized_X_train, normalized_X_val, normalized_X_test = normalized_X_train.drop(columns=['y']).values, normalized_X_val.drop(columns=['y']).values, normalized_X_test.drop(columns=['y']).values
    network = LGBM_w_Feature_Selector(model="mlp",problem_type="regression",param_grid=mlp_param_grid,X_train=normalized_X_train, X_test=normalized_X_test, slack=0.1, 
                                            X_val=normalized_X_val, y_val=normalized_y_val,y_train=normalized_y_train,y_test=normalized_y_test,iterations=10) 
    _,all_score, mo_score = network.feature_extraction(5, seed=42 , method="convergence", loss_tolerance=1,run_CV=True)
    all_scores.append(all_score)
    mo_scores.append(mo_score)

In [ ]:
### from this point on, experiments for GBMO for LGBM on M4 dailyall_scores = []
mo_scores = []
for i in range(200):
    data = pd.read_csv("/home/efe/Documents/m4-dataset/m4-preprocess/m4-preprocess/{}_M4_Daily.csv".format(str(i)))
    data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    #print(data.shape)
    train_data, X_test = data.iloc[:-50], data.iloc[-50:]
    nb = len(train_data)
    val_split = 0.3
    X_train, X_val = train_data.iloc[:int((1-val_split)*nb)], train_data.iloc[int((1-val_split)*nb):]
    y_train, y_val, y_test = X_train[["y"]], X_val[["y"]], X_test[["y"]]
    scaler = MinMaxScaler()
    normalized_X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    normalized_X_val = pd.DataFrame(scaler.fit_transform(X_val), columns=X_val.columns)
    normalized_X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)
    normalized_y_train = pd.DataFrame(scaler.fit_transform(y_train), columns=y_train.columns).values
    normalized_y_val = pd.DataFrame(scaler.fit_transform(y_val), columns=y_val.columns).values
    normalized_y_test = pd.DataFrame(scaler.fit_transform(y_test), columns=y_test.columns).values
    #X_train=(X_train-X_train.mean())/X_train.std()
    #X_val=(X_val-X_val.mean())/X_val.std()
    #X_test=(X_test-X_test.mean())/X_test.std()
    #y_train=(y_train-y_train.mean())/y_train.std()
    #y_val=(y_val-y_val.mean())/y_val.std()
    #y_test=(y_test-y_test.mean())/y_test.std()
    normalized_X_train, normalized_X_val, normalized_X_test = normalized_X_train.drop(columns=['y']).values, normalized_X_val.drop(columns=['y']).values, normalized_X_test.drop(columns=['y']).values
    network = LGBM_w_Feature_Selector(model="mlp",problem_type="regression",param_grid=mlp_param_grid,X_train=normalized_X_train, X_test=normalized_X_test, slack=0.1, 
                                            X_val=normalized_X_val, y_val=normalized_y_val,y_train=normalized_y_train,y_test=normalized_y_test,iterations=10) 
    _,all_score, mo_score = network.feature_extraction(5, seed=42 , method="convergence", loss_tolerance=1,run_CV=True)
    all_scores.append(all_score)
    mo_scores.append(mo_score)

In [ ]:
### try for 3-15 features
all, gb, mo, rfe = {}, {}, {}, {}

for f_number in [3, 5, 8, 12]:
    all_scores = []
    gb_scores = []
    mo_scores = []
    rfe_scores = []
    #f_number = 10
    for i in range(200):
        data = pd.read_csv("/home/efe/Documents/m4-dataset/m4-preprocess/m4-preprocess/{}_M4_Hourly.csv".format(str(i)))
        data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
        #print(data.shape)
        train_data, X_test = data.iloc[:-50], data.iloc[-50:]
        nb = len(train_data)
        val_split = 0.2
        X_train, X_val = train_data.iloc[:int((1-val_split)*nb)], train_data.iloc[int((1-val_split)*nb):]
        y_train, y_val, y_test = X_train[["y"]], X_val[["y"]], X_test[["y"]]
        scaler = MinMaxScaler()
        normalized_X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
        normalized_X_val = pd.DataFrame(scaler.fit_transform(X_val), columns=X_val.columns)
        normalized_X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)
        normalized_y_train = pd.DataFrame(scaler.fit_transform(y_train), columns=y_train.columns).values
        normalized_y_val = pd.DataFrame(scaler.fit_transform(y_val), columns=y_val.columns).values
        normalized_y_test = pd.DataFrame(scaler.fit_transform(y_test), columns=y_test.columns).values
        #X_train=(X_train-X_train.mean())/X_train.std()
        #X_val=(X_val-X_val.mean())/X_val.std()
        #X_test=(X_test-X_test.mean())/X_test.std()
        #y_train=(y_train-y_train.mean())/y_train.std()
        #y_val=(y_val-y_val.mean())/y_val.std()
        #y_test=(y_test-y_test.mean())/y_test.std()
        normalized_X_train, normalized_X_val, normalized_X_test = normalized_X_train.drop(columns=['y']).values, normalized_X_val.drop(columns=['y']).values, normalized_X_test.drop(columns=['y']).values
        network = LGBM_w_Feature_Selector(model="lgbm",problem_type="regression",param_grid=lgbm_param_grid,X_train=normalized_X_train, X_test=normalized_X_test, slack=0.1, 
                                                X_val=normalized_X_val, y_val=normalized_y_val,y_train=normalized_y_train,y_test=normalized_y_test,iterations=10) 
        _,all_score, gb_score, mo_score, rfe_score = network.feature_extraction(f_number, seed=42 , method="number of features", run_CV=True, include_RFE=True)
        all_scores.append(all_score)
        gb_scores.append(gb_score)    
        mo_scores.append(mo_score)
        rfe_scores.append(rfe_score)

all[f_number] = np.mean(all_scores)
gb[f_number] = np.mean(gb_scores)
mo[f_number] = np.mean(mo_scores)
rfe[f_number] = np.mean(rfe_scores)

### from this point on, experiments for FLBMO for MLP on M4 daily

In [ ]:
### from this point on, experiments for FLBMO for L### try for 3-15 features
all, gb, mo = {}, {}, {}

for f_number in [3, 5, 8, 12]:
    all_scores = []
    gb_scores = []
    mo_scores = []
    #f_number = 10
    for i in range(200):
        data = pd.read_csv("/home/efe/Documents/m4-dataset/m4-preprocess/m4-preprocess/{}_M4_Daily.csv".format(str(i)))
        data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
        #print(data.shape)
        train_data, X_test = data.iloc[:-50], data.iloc[-50:]
        nb = len(train_data)
        val_split = 0.2
        X_train, X_val = train_data.iloc[:int((1-val_split)*nb)], train_data.iloc[int((1-val_split)*nb):]
        y_train, y_val, y_test = X_train[["y"]], X_val[["y"]], X_test[["y"]]
        scaler = MinMaxScaler()
        normalized_X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
        normalized_X_val = pd.DataFrame(scaler.fit_transform(X_val), columns=X_val.columns)
        normalized_X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)
        normalized_y_train = pd.DataFrame(scaler.fit_transform(y_train), columns=y_train.columns).values
        normalized_y_val = pd.DataFrame(scaler.fit_transform(y_val), columns=y_val.columns).values
        normalized_y_test = pd.DataFrame(scaler.fit_transform(y_test), columns=y_test.columns).values
        #X_train=(X_train-X_train.mean())/X_train.std()
        #X_val=(X_val-X_val.mean())/X_val.std()
        #X_test=(X_test-X_test.mean())/X_test.std()
        #y_train=(y_train-y_train.mean())/y_train.std()
        #y_val=(y_val-y_val.mean())/y_val.std()
        #y_test=(y_test-y_test.mean())/y_test.std()
        normalized_X_train, normalized_X_val, normalized_X_test = normalized_X_train.drop(columns=['y']).values, normalized_X_val.drop(columns=['y']).values, normalized_X_test.drop(columns=['y']).values
        network = LGBM_w_Feature_Selector(model="mlp",problem_type="regression",param_grid=mlp_param_grid,X_train=normalized_X_train, X_test=normalized_X_test, slack=0.1, 
                                                X_val=normalized_X_val, y_val=normalized_y_val,y_train=normalized_y_train,y_test=normalized_y_test,iterations=10) 
        _,all_score, gb_score, mo_score = network.feature_extraction(f_number, seed=42 , method="number of features", run_CV=True)
        all_scores.append(all_score)
        gb_scores.append(gb_score)    
        mo_scores.append(mo_score)
    all[f_number] = np.mean(all_scores)
    gb[f_number] = np.mean(gb_scores)
    mo[f_number] = np.mean(mo_scores)


In [ ]:
### from this point on, experiments for GBMO for MLP on M4 quart

In [ ]:
all_scores = []
gb_scores= []
mo_scores= []
for i in range(100):
    data = pd.read_csv("/home/efe/Documents/m4-dataset/m4-preprocess/m4-preprocess/{}_M4_Quarterly.csv".format(str(i)))
    data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    #print(data.shape)
    nb = len(data)
    val_split = 0.3
    test_split = 0.1
    X_train, X_val, X_test = data.iloc[:int((1-(val_split+test_split))*nb)], data.iloc[int((1-(val_split+test_split))*nb):int((1-(test_split))*nb)], data.iloc[int((1-(test_split))*nb):]
    y_train, y_val, y_test = X_train["y"], X_val["y"], X_test["y"]
    X_train=(X_train-X_train.mean())/X_train.std()
    X_val=(X_val-X_val.mean())/X_val.std()
    X_test=(X_test-X_test.mean())/X_test.std()
    y_train=(y_train-y_train.mean())/y_train.std()
    y_val=(y_val-y_val.mean())/y_val.std()
    y_test=(y_test-y_test.mean())/y_test.std()
    X_train, X_val, X_test = X_train.drop(columns=['y']).values, X_val.drop(columns=['y']).values, X_test.drop(columns=['y']).values
    network = LGBM_w_Feature_Selector(model="mlp",problem_type="regression", max_depth=3, boosting_type="gbdt", X_train=X_train, X_test=X_test, slack=0.05, 
                                            X_val=X_val, y_val=y_val,y_train=y_train,y_test=y_test,iterations=10,param_grid=mlp_param_grid) 
    _,all_score, gb_score, mo_score = network.feature_extraction(f_number, seed=42 , method="number of features",run_CV=True)
    all_scores.append(all_score)
    gb_scores.append(gb_score)    
    mo_scores.append(mo_score)
all= np.mean(all_scores)
gb= np.mean(gb_scores)
mo= np.mean(mo_scores)

In [ ]:
all

In [ ]:
gb

In [ ]:
mo

In [ ]:
### from this point on, experiments for GBMO for MLP on M4 quart

In [ ]:
all_scores = []
gb_scores= []
mo_scores= []
rfe_scores=[]
for i in range(100):
    data = pd.read_csv("/home/efe/Documents/m4-dataset/m4-preprocess/m4-preprocess/{}_M4_Quarterly.csv".format(str(i)))
    data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    #print(data.shape)
    nb = len(data)
    val_split = 0.3
    test_split = 0.1
    X_train, X_val, X_test = data.iloc[:int((1-(val_split+test_split))*nb)], data.iloc[int((1-(val_split+test_split))*nb):int((1-(test_split))*nb)], data.iloc[int((1-(test_split))*nb):]
    y_train, y_val, y_test = X_train["y"], X_val["y"], X_test["y"]
    X_train=(X_train-X_train.mean())/X_train.std()
    X_val=(X_val-X_val.mean())/X_val.std()
    X_test=(X_test-X_test.mean())/X_test.std()
    y_train=(y_train-y_train.mean())/y_train.std()
    y_val=(y_val-y_val.mean())/y_val.std()
    y_test=(y_test-y_test.mean())/y_test.std()
    X_train, X_val, X_test = X_train.drop(columns=['y']).values, X_val.drop(columns=['y']).values, X_test.drop(columns=['y']).values
    network = LGBM_w_Feature_Selector(model="mlp",problem_type="regression", max_depth=3, boosting_type="gbdt", X_train=X_train, X_test=X_test, slack=0.05, 
                                            X_val=X_val, y_val=y_val,y_train=y_train,y_test=y_test,iterations=10,param_grid=mlp_param_grid) 
    _,all_score, gb_score, mo_score = network.feature_extraction(f_number, seed=42 , method="number of features",run_CV=True)
    all_scores.append(all_score)
    gb_scores.append(gb_score)    
    mo_scores.append(mo_score)
all= np.mean(all_scores)
gb = np.mean(gb_scores)
mo = np.mean(mo_scores)

In [ ]:
all

In [ ]:
gb

In [ ]:
mo

In [ ]:
### from this point on, experiments for FLBMO for MLP on M4 quart

In [ ]:
all, gb, mo, rfe = {}, {}, {}, {}
for f_number in [3, 5, 8, 12]:
    all_scores = []
    gb_scores = []
    mo_scores = []
    rfe_scores = []
    #f_number = 10
    for i in range(100):
        data = pd.read_csv("/home/efe/Documents/m4-dataset/m4-preprocess/m4-preprocess/{}_M4_Quarterly.csv".format(str(i)))
        data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
        #print(data.shape)
        nb = len(data)
        val_split = 0.3
        test_split = 0.1
        X_train, X_val, X_test = data.iloc[:int((1-(val_split+test_split))*nb)], data.iloc[int((1-(val_split+test_split))*nb):int((1-(test_split))*nb)], data.iloc[int((1-(test_split))*nb):]
        y_train, y_val, y_test = X_train["y"], X_val["y"], X_test["y"]
        X_train=(X_train-X_train.mean())/X_train.std()
        X_val=(X_val-X_val.mean())/X_val.std()
        X_test=(X_test-X_test.mean())/X_test.std()
        y_train=(y_train-y_train.mean())/y_train.std()
        y_val=(y_val-y_val.mean())/y_val.std()
        y_test=(y_test-y_test.mean())/y_test.std()
        X_train, X_val, X_test = X_train.drop(columns=['y']).values, X_val.drop(columns=['y']).values, X_test.drop(columns=['y']).values
        network = LGBM_w_Feature_Selector(model="lgbm",problem_type="regression", max_depth=3, boosting_type="gbdt", X_train=X_train, X_test=X_test, slack=0.05, 
                                                X_val=X_val, y_val=y_val,y_train=y_train,y_test=y_test,iterations=10,param_grid=lgbm_aram_grid) 
        _,all_score, gb_score, mo_score, rfe_score = network.feature_extraction(f_number, seed=42 , method="number of features",run_CV=True)
        all_scores.append(all_score)
        gb_scores.append(gb_score)    
        mo_scores.append(mo_score)
        rfe_scores.append(rfe_score)

all[f_number] = np.mean(all_scores)
gb[f_number] = np.mean(gb_scores)
mo[f_number] = np.mean(mo_scores)
rfe[f_number] = np.mean(rfe_scores)

In [ ]:
all

In [ ]:
gb

In [ ]:
rfe

In [ ]:
mo

In [ ]:
### from this point on, experiments for FLBMO for MLP on M4 quart

In [ ]:
all, gb, mo = {}, {}, {}
for f_number in [3, 5, 8, 12]:
    all_scores = []
    gb_scores = []
    mo_scores = []
    #f_number = 10
    for i in range(100):
        data = pd.read_csv("/home/efe/Documents/m4-dataset/m4-preprocess/m4-preprocess/{}_M4_Quarterly.csv".format(str(i)))
        data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
        #print(data.shape)
        nb = len(data)
        val_split = 0.3
        test_split = 0.1
        X_train, X_val, X_test = data.iloc[:int((1-(val_split+test_split))*nb)], data.iloc[int((1-(val_split+test_split))*nb):int((1-(test_split))*nb)], data.iloc[int((1-(test_split))*nb):]
        y_train, y_val, y_test = X_train["y"], X_val["y"], X_test["y"]
        X_train=(X_train-X_train.mean())/X_train.std()
        X_val=(X_val-X_val.mean())/X_val.std()
        X_test=(X_test-X_test.mean())/X_test.std()
        y_train=(y_train-y_train.mean())/y_train.std()
        y_val=(y_val-y_val.mean())/y_val.std()
        y_test=(y_test-y_test.mean())/y_test.std()
        X_train, X_val, X_test = X_train.drop(columns=['y']).values, X_val.drop(columns=['y']).values, X_test.drop(columns=['y']).values
        network = LGBM_w_Feature_Selector(model="lgbm",problem_type="regression", max_depth=3, boosting_type="gbdt", X_train=X_train, X_test=X_test, slack=0.05, 
                                                X_val=X_val, y_val=y_val,y_train=y_train,y_test=y_test,iterations=10,param_grid=param_grid) 
        _,all_score, gb_score, mo_score = network.feature_extraction(f_number, seed=42 , method="number of features",run_CV=True)
        all_scores.append(all_score)
        gb_scores.append(gb_score)    
        mo_scores.append(mo_score)
    all[f_number] = np.mean(all_scores)
    gb[f_number] = np.mean(gb_scores)
    mo[f_number] = np.mean(mo_scores)

In [ ]:
all

In [ ]:
gb

In [ ]:
mo